# Lab : Image Classification using Convolutional Neural Networks

At the end of this laboratory, you would get familiarized with

*   Creating deep networks using Keras
*   Steps necessary in training a neural network
*   Prediction and performance analysis using neural networks

---

# **In case you use a colaboratory environment**
By default, Colab notebooks run on CPU.
You can switch your notebook to run with GPU.

In order to obtain access to the GPU, you need to choose the tab Runtime and then select “Change runtime type” as shown in the following figure:

![Changing runtime](https://miro.medium.com/max/747/1*euE7nGZ0uJQcgvkpgvkoQg.png)

When a pop-up window appears select GPU. Ensure “Hardware accelerator” is set to GPU.

# **Working with a new dataset: CIFAR-10**

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. More information about CIFAR-10 can be found [here](https://www.cs.toronto.edu/~kriz/cifar.html).

In Keras, the CIFAR-10 dataset is also preloaded in the form of four Numpy arrays. x_train and y_train contain the training set, while x_test and y_test contain the test data. The images are encoded as Numpy arrays and their corresponding labels ranging from 0 to 9.

Your task is to:

*   Visualize the images in CIFAR-10 dataset. Create a 10 x 10 plot showing 10 random samples from each class.
*   Convert the labels to one-hot encoded form.
*   Normalize the images.




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from keras.backend import clear_session

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [ ]:
# Your code here :
#Visualize CIFAR-10 Images: 10 x 10 Grid (10 random samples per class)
num_classes = 10
samples_per_class = 10
plt.figure(figsize=(10,10))
for c in range(num_classes):
    # Get indices of all images in class c
    idxs = np.where(y_train.flatten() == c)[0]
    # Randomly select 10 indices for class c
    selected_idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(selected_idxs):
        plt_idx = c * samples_per_class + i + 1
        plt.subplot(num_classes, samples_per_class, plt_idx)
        plt.imshow(x_train[idx])
        plt.axis('off')
        if i == 0:
            plt.ylabel(f"Class {c}", fontsize=12)
plt.suptitle("CIFAR-10: 10 Random Samples per Class", fontsize=16)
plt.show()

In [ ]:
#Preprocess Data: One-Hot Encode Labels and Normalize Images
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

## Define the following model (same as the one in tutorial)

For the convolutional front-end, start with a single convolutional layer with a small filter size (3,3) and a modest number of filters (32) followed by a max pooling layer. 

Use the input as (32,32,3). 

The filter maps can then be flattened to provide features to the classifier. 

Use a dense layer with 100 units before the classification layer (which is also a dense layer with softmax activation).

In [5]:
from keras.backend import clear_session
clear_session()

In [ ]:
# Your code here :
#Define, Compile & Train a Simple CNN Model
clear_session()  # Clear previous models

simple_model = Sequential([
    InputLayer(input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(num_classes, activation='softmax')
])


*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 50 epochs with a batch size of 512.

In [ ]:
simple_model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])
print(simple_model.summary())

# Train the simple model for 50 epochs with batch size 512
history_simple = simple_model.fit(x_train, y_train_cat, epochs=50, batch_size=512, 
                                  validation_data=(x_test, y_test_cat))


*   Plot the cross entropy loss curve and the accuracy curve

In [ ]:
# Your code here :
# Plot loss and accuracy curves
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history_simple.history['loss'], label='Train Loss')
plt.plot(history_simple.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Simple Model Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history_simple.history['accuracy'], label='Train Accuracy')
plt.plot(history_simple.history['val_accuracy'], label='Val Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Simple Model Accuracy")
plt.legend()
plt.show()

## Defining Deeper Architectures: VGG Models

*   Define a deeper model architecture for CIFAR-10 dataset and train the new model for 50 epochs with a batch size of 512. We will use VGG model as the architecture.

Stack two convolutional layers with 32 filters, each of 3 x 3. 

Use a max pooling layer and next flatten the output of the previous layer and add a dense layer with 128 units before the classification layer. 

For all the layers, use ReLU activation function. 

Use same padding for the layers to ensure that the height and width of each layer output matches the input


In [9]:
from keras.backend import clear_session
clear_session()

In [ ]:
# Your code here :
vgg_model = Sequential([
    InputLayer(input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])


*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 50 epochs with a batch size of 512.

In [ ]:
# Your code here :
vgg_model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])
print(vgg_model.summary())

# Train the deeper model for 50 epochs with batch size 512
history_vgg = vgg_model.fit(x_train, y_train_cat, epochs=50, batch_size=512, 
                            validation_data=(x_test, y_test_cat))


*   Compare the performance of both the models by plotting the loss and accuracy curves of both the training steps. Does the deeper model perform better? Comment on the observation.
 

In [ ]:
# Evaluate both models on the test set
test_loss_simple, test_acc_simple = simple_model.evaluate(x_test, y_test_cat, verbose=0)
test_loss_vgg, test_acc_vgg = vgg_model.evaluate(x_test, y_test_cat, verbose=0)
print("Simple Model Test Accuracy:", test_acc_simple)
print("Deeper VGG-like Model Test Accuracy:", test_acc_vgg)


**Comment on the observation**

*(Double-click or enter to edit)*

...

*   Use predict function to predict the output for the test split
*   Plot the confusion matrix for the new model and comment on the class confusions.


In [ ]:
# Confusion Matrix for the Deeper Model
y_pred_vgg = vgg_model.predict(x_test)
y_pred_labels = np.argmax(y_pred_vgg, axis=1)
y_true_labels = y_test.flatten()
cm = confusion_matrix(y_true_labels, y_pred_labels)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Deeper VGG-like Model")
plt.show()

**Comment here :**

*(Double-click or enter to edit)*

...

*    Print the test accuracy for the trained model.

In [ ]:
# Your code here :
# Plot training curves for the deeper model
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history_vgg.history['loss'], label='Train Loss')
plt.plot(history_vgg.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Deeper VGG-like Model Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history_vgg.history['accuracy'], label='Train Accuracy')
plt.plot(history_vgg.history['val_accuracy'], label='Val Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Deeper VGG-like Model Accuracy")
plt.legend()
plt.show()


## Define the complete VGG architecture.

Stack two convolutional layers with 64 filters, each of 3 x 3 followed by max pooling layer. 

Stack two more convolutional layers with 128 filters, each of 3 x 3, followed by max pooling, followed by two more convolutional layers with 256 filters, each of 3 x 3, followed by max pooling. 

Flatten the output of the previous layer and add a dense layer with 128 units before the classification layer. 

For all the layers, use ReLU activation function. 

Use same padding for the layers to ensure that the height and width of each layer output matches the input

*   Change the size of input to 64 x 64.

In [15]:
from keras.backend import clear_session
clear_session()

In [ ]:
# Your code here :
# Resize images to 64x64
x_train_resized = tf.image.resize(x_train, [64,64]).numpy()
x_test_resized  = tf.image.resize(x_test, [64,64]).numpy()


*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 10 epochs with a batch size of 512.
*   Predict the output for the test split and plot the confusion matrix for the new model and comment on the class confusions.

In [ ]:
# Your code here :

vgg_complete = Sequential([
    InputLayer(input_shape=(64,64,3)),
    # First block: 2 Conv layers with 64 filters
    Conv2D(64, (3,3), activation='relu', padding='same'),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    # Second block: 2 Conv layers with 128 filters
    Conv2D(128, (3,3), activation='relu', padding='same'),
    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    # Third block: 2 Conv layers with 256 filters
    Conv2D(256, (3,3), activation='relu', padding='same'),
    Conv2D(256, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

vgg_complete.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])
print(vgg_complete.summary())

# Train the complete VGG model for 10 epochs with batch size 512
history_vgg_complete = vgg_complete.fit(x_train_resized, y_train_cat, epochs=10, batch_size=512, 
                                        validation_data=(x_test_resized, y_test_cat))


In [ ]:
# Plot confusion matrix for the complete VGG model
y_pred_complete = vgg_complete.predict(x_test_resized)
y_pred_complete_labels = np.argmax(y_pred_complete, axis=1)
cm_complete = confusion_matrix(y_test.flatten(), y_pred_complete_labels)
plt.figure(figsize=(8,6))
sns.heatmap(cm_complete, annot=True, fmt="d", cmap="Greens",
            xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Complete VGG Model")
plt.show()

In [ ]:
# Evaluate complete VGG model on the test set
test_loss_complete, test_acc_complete = vgg_complete.evaluate(x_test_resized, y_test_cat, verbose=0)
print("Complete VGG Model Test Accuracy:", test_acc_complete)


# Understanding deep networks

*   What is the use of activation functions in network? Why is it needed?
*   We have used softmax activation function in the exercise. There are other activation functions available too. What is the difference between sigmoid activation and softmax activation?
*   What is the difference between categorical crossentropy and binary crossentropy loss?

**Write the answers below :**

1 - Use of activation functions:

Activation functions introduce non-linearity into the network, allowing it to learn and model complex patterns.

2 - Key Differences between sigmoid and softmax:

Sigmoid activation outputs independent probabilities between 0 and 1 for each neuron, suitable for binary classification. Softmax activation outputs a probability distribution over multiple classes where the probabilities sum to 1, making it ideal for multi-class classification.


3 - Key Differences between categorical crossentropy and binary crossentropy loss:

Categorical crossentropy is used for multi-class classification with mutually exclusive classes, while binary crossentropy is used for binary classification or multi-label tasks where each label is independent.
